# GeneriCorp Internal Tools

## Notebook Setup

In [1]:
# Importing the necessary Python libraries
import os
import json
import pandas as pd
from langchain.output_parsers import CommaSeparatedListOutputParser
from langchain_core.prompts import PromptTemplate
from langchain_openai import ChatOpenAI

In [2]:
# Instantiating the chat model
chat_model = ChatOpenAI(api_key = os.environ['OPENAI_API_KEY'],
                        model = 'gpt-4o-mini')

In [3]:
# Loading the company history from file
with open('../../genericorp/about-gc/company-history.md', 'r', encoding = 'utf-8') as file:
    company_history = file.read()

In [4]:
# Loading the list of GeneriCorp tools and descriptions from file
with open('genericorp_internal_tools.json', 'r') as file:
    genericorp_internal_tools = json.load(file)

## Generating Knowledge Item Topic Ideas

In [5]:
# Creating the topic generation prompt
TOPIC_GENERATION_PROMPT = '''Imagine a fictional company called GeneriCorp. Their company history is as follows:

{company_history}

Please generate a list of potential knowledge items that might support the internal tool named "{tool_name}". Here is a description of the tool:

{tool_description} 

Consider the tool's purpose, features, and how it integrates within GeneriCorp's operations.

Please output the knowledge items in a Python comma-separated list format. Do not provide any additional text aside from the Python list.
'''


# Instantiating the output parser
output_parser = CommaSeparatedListOutputParser()

# Setting up the chat prompt template
topic_generation_prompt_template = PromptTemplate(
    template = TOPIC_GENERATION_PROMPT + '\n{format_instructions}',
    input_variables = ['company_history', 'tool_name', 'tool_description'],
    partial_variables = {'format_instructions': output_parser.get_format_instructions()}
)

# Creating the topic generation chain
topic_generation_chain = topic_generation_prompt_template | chat_model | output_parser

In [6]:
# Setting up the cache directory
TOPICS_CACHE_DIR = 'knowledge_topics_cache'
os.makedirs(TOPICS_CACHE_DIR, exist_ok = True)

# Dictionary to store all knowledge topics
all_knowledge_topics = {}

# Iterating through each tool
for tool_name, tool_description in genericorp_internal_tools.items():

    # Creating a cache filename for this specific tool
    cache_file = f"{TOPICS_CACHE_DIR}/{tool_name.replace(' ', '_').lower()}_topics.json"
    
    # Checking if cached topics exist for this tool
    if os.path.exists(cache_file):

        # Loading topics from cache
        with open(cache_file, 'r') as f:
            tool_topics = json.load(f)
    else:
        # Generating topics using the AI model for this tool
        tool_topics = topic_generation_chain.invoke({
            "company_history": company_history,
            "tool_name": tool_name,
            "tool_description": tool_description
        })
        
        # Caching the results
        with open(cache_file, 'w') as f:
            json.dump(tool_topics, f)
    
    # Store topics for this tool
    all_knowledge_topics[tool_name] = tool_topics
    
    # Print results for this tool
    print(f"\nTool: {tool_name}")
    print(f"Number of topics: {len(tool_topics)}")
    print("Topics:", tool_topics)


Tool: GenOps Suite
Number of topics: 15
Topics: ['`HR management best practices', 'finance workflow automation guidelines', 'procurement process optimization', 'logistics and supply chain tracking techniques', 'internal ticketing system usage', 'standard operating procedures for GenOps Suite', 'troubleshooting common issues in GenOps', 'integration methods with existing systems', 'data management and reporting techniques', 'user training resources for employees', 'case studies on workflow automation success', 'compliance and regulatory requirements', 'cost-saving strategies in operations', 'performance metrics for enterprise operations', 'FAQs about the GenOps Suite`']

Tool: CircuitForge
Number of topics: 15
Topics: ['`user_manual', 'circuit_design_guidelines', 'simulation_test_cases', 'AI_diagnostics_integration', 'debugging_techniques', 'hardware_component_library', 'firmware_update_protocols', 'troubleshooting_guide', 'performance_metrics_analysis', 'circuit_schematic_examples', '

## Generating the Handbook Pages from the Topics

In [7]:
# # Creating the handbook page generation prompt
# HANDBOOK_PAGE_GENERATION_PROMPT = '''Generate a professional and structured employee handbook page on the topic:

# {handbook_topic}

# As a reminder, here is the company history:

# {company_history}

# The handbook page should be formatted with clear headings and subheadings, using a formal yet approachable tone suitable for employees.

# The page should include the following structured sections:
# 	1.	Title – A clear and precise title reflecting the topic.
# 	2.	Introduction – A brief overview explaining why this topic is important and its relevance to employees.
# 	3.	Policy Overview (if applicable) – A structured breakdown of the rules, guidelines, or best practices employees must follow.
# 	4.	Employee Responsibilities – Expectations for employees regarding this topic.
# 	5.	Company Commitment (if applicable) – How the company supports or enforces this policy (e.g., for Diversity & Inclusion, this might include training programs; for Workplace Safety, this might cover compliance measures).
# 	6.	Procedures – Steps employees need to follow (e.g., how to report an issue, request a benefit, or participate in a program).
# 	7.	Additional Notes & Compliance – Any legal, ethical, or procedural considerations employees should be aware of.
# 	8.	Resources & Contact Information – Who employees should reach out to for more information (e.g., HR, department heads, internal portals).

# Ensure that the page is concise, clear, and aligned with best HR practices. If applicable, provide bullet points for readability and avoid overly technical language while maintaining professionalism.

# Format the output as a well-structured, ready-to-use employee handbook page.
# '''

# # Setting up the chat prompt template
# handbook_page_prompt_template = PromptTemplate(
#     template = HANDBOOK_PAGE_GENERATION_PROMPT,
#     input_variables = ['handbook_topic', 'company_history'],
# )

# # Creating the handbook page generation chain
# handbook_page_generation_chain = handbook_page_prompt_template | chat_model

In [8]:
# # Setting the directory for the employee handbook pages
# employee_handbook_dir = '../../genericorp/employee-handbook/'

# # Iterating over each of the topics for the employee handbook
# for handbook_topic in handbook_topics:
    
#     # Defining the file path for the employee handbook page
#     file_path = f'{employee_handbook_dir}{handbook_topic}.md'
    
#     # Checking if file for the employee handbook page already exists
#     if not os.path.exists(file_path):

#         # Generating the employee handbook page with the LLM
#         response = handbook_page_generation_chain.invoke({"handbook_topic": handbook_topic, "company_history": company_history})
        
#         # Saving the employee handbook page to file
#         with open(file_path, 'w', encoding = 'utf-8') as file:
#             file.write(response.content)